In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Dataset Preprocessing

Working on dataset obtained from https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data?select=train.csv.zip

This is a labelled dataset.

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'/content/gdrive/MyDrive/train.csv')

In [ ]:

df['total'] = df['insult'] + df['threat'] + df['toxic'] + df['severe_toxic'] + df['obscene'] + df['identity_hate'] 

In [ ]:
df['total']

0         0
1         0
2         0
3         0
4         0
         ..
159566    0
159567    0
159568    0
159569    0
159570    0
Name: total, Length: 159571, dtype: int64

In [ ]:
# original dataset is highly biased
df['total'].value_counts()

0    143346
1      6360
3      4209
2      3480
4      1760
5       385
6        31
Name: total, dtype: int64

### Limiting the imbalance (all column labels = 0) to 25000 entries instead of 143346 for better results.

In [ ]:
count = 0
for i in range(len(df)):
  if df['total'][i] == 0 and count <118346:
    df = df.drop(axis = 0, index = i)
    count = count + 1
  if i % 1000 == 0:
    print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000


KeyboardInterrupt: ignored

In [ ]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,total
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,4
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0,1
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0,1
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,4
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,3
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,0


In [ ]:
df['total'].value_counts()

0    25000
1     6360
3     4209
2     3480
4     1760
5      385
6       31
Name: total, dtype: int64

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.to_csv('/content/gdrive/MyDrive/balanced_train.csv', index = False)

In [ ]:
# Reading the balance train dataset - 
df = pd.read_csv('/content/gdrive/MyDrive/balanced_train.csv')

# Find embeddings

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
# roberta = SentenceTransformer('sentence-transformers/stsb-roberta-large')

In [ ]:
lst_embeddings = []

In [ ]:
import pandas as pd
df = pd.read_csv(r'/content/gdrive/MyDrive/balanced_train.csv')

In [ ]:
for idx in range(len(df['comment_text'])):
  if idx%1000==0:
    print(idx)
  sentences_embeddings = sbert_model.encode(df['comment_text'][idx])
  lst_embeddings.append(sentences_embeddings)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000


In [ ]:
df['vectors'] = lst_embeddings

In [ ]:
df.to_csv('/content/gdrive/MyDrive/jigsaw_25000_bert_encoding.csv')

In [ ]:
# df

In [ ]:
import pandas as pd
import numpy as np
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')
df1=pd.read_csv('/content/gdrive/MyDrive/jigsaw_25000_bert_encoding.csv',converters={'vectors':converter})

#df1=pd.read_csv('/content/gdrive/MyDrive/roberta_encoded.csv',converters={'vectors':converter})

In [ ]:
type(df1['vectors'][1])

numpy.ndarray

## **Support Vector Machine Implementation - Classification**

For target labels - 'toxic' and 'identity_hate'

In [ ]:
from sklearn import svm
model_identity_hate_lr = None
model_toxic_lr = None

import copy




def baseline_ml(target, df):
  X = df['vectors']
  print(type(X[1]))
  y = df[target]
  #print(y)
  print("SVM for --- ",target)
  lst = []
  for i in X:
    lst.append(i)
  print(type(lst[2]))

  from sklearn.model_selection import train_test_split
  x_train, x_test, y_train, y_test = train_test_split(lst, list(y), test_size=0.25, random_state=0)

  model = svm.SVC()
  model.fit(x_train, y_train)
  #Prediction using Logistic Reg
  if target == 'toxic':
    model_toxic_lr = copy.deepcopy(model)
    
  elif target == 'identity_hate':
    model_identity_hate_lr = copy.deepcopy(model)

  predictions = model.predict(x_test)
  
  #To improve recall
  # proba = model.predict_proba(x_test)
  # if target == 'toxic':
  #   model_toxic_lr = copy.deepcopy(model)
  # elif target == 'identity_hate':
  #   model_identity_hate_lr = copy.deepcopy(model)
  # predictions = []
  # for i in proba:
  #   if i[0] > 0.7:
  #     predictions.append(1)
  #   else:
  #     predictions.append(0)


  import matplotlib.pyplot as plt
  import seaborn as sns
  from sklearn import metrics
  cm = metrics.confusion_matrix(y_test, predictions)
  print(cm)
  precision = metrics.precision_score(y_test, predictions)
  recall = metrics.recall_score(y_test, predictions)
  F1 = 2 * (precision * recall) / (precision + recall)

  print("Accuracy of {} -- {}".format(target, metrics.accuracy_score(y_test, predictions)))
  print("Precision:",precision)
  print("Recall:",recall)
  print("F1 Score - ",F1)

  print("==========")
  print()

  return model

In [ ]:
model_toxic_lr = baseline_ml('toxic', df1)
model_identity_hate_lr = baseline_ml('identity_hate', df1)

<class 'numpy.ndarray'>
SVM for ---  toxic
<class 'numpy.ndarray'>
[[5992  456]
 [ 565 3294]]
Accuracy of toxic -- 0.9009411079848647
Precision: 0.8784
Recall: 0.85358901269759
F1 Score -  0.8658167958995925

<class 'numpy.ndarray'>
SVM for ---  identity_hate
<class 'numpy.ndarray'>
[[9945   12]
 [ 290   60]]
Accuracy of identity_hate -- 0.9706995245949355
Precision: 0.8333333333333334
Recall: 0.17142857142857143
F1 Score -  0.28436018957345977



## Predicting on unlabelled and unseen dataset

Tagged.csv - Data scraped from wikipedia(unbiased) and google(biased)

In [ ]:
df_tagged = pd.read_csv('/content/gdrive/MyDrive/tagged.csv')

In [ ]:
df_tagged
df_temp = pd.DataFrame()
l_s = []
l_pred = []
for i in range(len(df_tagged['Sentences'])):
  embs = sbert_model.encode(df_tagged['Sentences'][i])
  prediction = model_toxic_lr.predict(embs.reshape(1,-1))  
  l_s.append(df_tagged['Sentences'][i])
  l_pred.append(prediction[0])


In [ ]:
df_temp['sent'] = l_s
df_temp['preds'] = l_pred

In [ ]:
df_temp

,sent,preds
0,'Real Housewives of Miami' reboot includes fra...,0
1,Reserve Bank Puts Off Plunging Country Into In...,0
2,A Latina scientist co-created a new Covid vacc...,0
3,Ayala: Democratic Latino voters may be agonizi...,0
4,India Defense Ministry denies clearance to fil...,0
...,...,...
3613,The only way you're going to get Junior to vis...,0
3614,Sen. Kelly has apparently decided that the sen...,0
3615,What would you expect from this book?,0
3616,"""And what this involved was a steady campaign ...",0


In [ ]:
df_temp.to_csv('/content/gdrive/MyDrive/predicted_google_wiki_bert_encoding.csv')

### Filtering out sentences that are predicted toxic in nature that is preds = 1.

In [ ]:
df_temp = pd.read_csv('/content/gdrive/MyDrive/predicted_google_wiki_bert_encoding.csv')
list_sentences = []
for i in range(len(df_temp['preds'])):
  if df_temp['preds'][i] == 1:
    list_sentences.append(df_temp['sent'][i])

In [ ]:
len(list_sentences)


555

In [ ]:
list_biased = ['gay','lesbian','black','transgender','chinese']

offensive_word_emb = sbert_model.encode(list_biased)

In [ ]:
lst_embeddings = []
for idx in range(len(df_temp['sent'])):
  if idx%1000==0:
    print(idx)
  sentences_embeddings = sbert_model.encode(df_temp['sent'][idx])
  lst_embeddings.append(sentences_embeddings)

0
1000
2000
3000


In [ ]:
len(lst_embeddings)

3618

In [ ]:
df_temp['vectors'] = lst_embeddings

# BERT preditciont files

In [ ]:
df_temp.to_csv('/content/gdrive/MyDrive/predicted_google_wiki_bert_encoding.csv', index = False)

In [ ]:
df_temp

,Unnamed: 0,sent,preds,vectors
0,0,'Real Housewives of Miami' reboot includes fra...,0,"[-0.54321486, 0.3058867, 0.11035681, 0.0978671..."
1,1,Reserve Bank Puts Off Plunging Country Into In...,0,"[-0.7183743, 0.027845453, 0.6962283, 0.4185731..."
2,2,A Latina scientist co-created a new Covid vacc...,0,"[-0.5572357, 0.5703229, 0.19752829, 0.02539971..."
3,3,Ayala: Democratic Latino voters may be agonizi...,0,"[-0.61914414, -0.13753335, -0.21173638, 0.0923..."
4,4,India Defense Ministry denies clearance to fil...,0,"[0.38713986, 0.76206887, 0.40441874, 0.3612578..."
...,...,...,...,...
3613,3613,The only way you're going to get Junior to vis...,0,"[-0.0077593676, 0.86078084, 1.2438513, 1.17803..."
3614,3614,Sen. Kelly has apparently decided that the sen...,0,"[-0.15648964, 0.56082344, 0.72321916, -0.02952..."
3615,3615,What would you expect from this book?,0,"[-0.08646081, -0.8799631, 1.0081621, 0.0905289..."
3616,3616,"""And what this involved was a steady campaign ...",0,"[-0.13292979, 0.38845474, 0.18564925, 0.178709..."


In [ ]:
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')
df1=pd.read_csv('/content/gdrive/MyDrive/predicted_google_wiki_bert_encoding.csv',converters={'vectors':converter})

In [ ]:
df1

,Unnamed: 0,sent,preds,vectors
0,0,'Real Housewives of Miami' reboot includes fra...,0,"[-0.543214858, 0.305886686, 0.110356808, 0.097..."
1,1,Reserve Bank Puts Off Plunging Country Into In...,0,"[-0.718374312, 0.0278454535, 0.696228325, 0.41..."
2,2,A Latina scientist co-created a new Covid vacc...,0,"[-0.557235718, 0.570322871, 0.197528288, 0.025..."
3,3,Ayala: Democratic Latino voters may be agonizi...,0,"[-0.619144142, -0.137533352, -0.211736381, 0.0..."
4,4,India Defense Ministry denies clearance to fil...,0,"[0.387139857, 0.762068868, 0.404418737, 0.3612..."
...,...,...,...,...
3613,3613,The only way you're going to get Junior to vis...,0,"[-0.00775936758, 0.860780835, 1.2438513, 1.178..."
3614,3614,Sen. Kelly has apparently decided that the sen...,0,"[-0.15648964, 0.560823441, 0.723219156, -0.029..."
3615,3615,What would you expect from this book?,0,"[-0.0864608064, -0.8799631, 1.00816214, 0.0905..."
3616,3616,"""And what this involved was a steady campaign ...",0,"[-0.132929787, 0.388454735, 0.185649246, 0.178..."


In [ ]:
list_sentences = []
list_vectors = []
for i in range(len(df_temp['preds'])):
  if df_temp['preds'][i] == 1:
    list_sentences.append(df_temp['sent'][i])
    list_vectors.append(df_temp['vectors'][i])

In [ ]:

from sklearn.metrics.pairwise import cosine_similarity

new_df = pd.DataFrame()
new_df['sentences'] = list_sentences
new_df['vectors'] = list_vectors
for i in list_biased:
  new_df[i] = ''
for each_vector in range(len(df_temp['vectors'])):
  for each_Word in range(len(offensive_word_emb)):
    cosine = cosine_similarity(offensive_word_emb[each_Word].reshape(1,-1), df_temp['vectors'][each_vector].reshape(1,-1))

    if cosine > 0.3:
      new_df[list_biased[each_Word]][each_vector] = 1
    else:
      new_df[list_biased[each_Word]][each_vector] = 0

In [ ]:
new_df.to_csv("/content/gdrive/MyDrive/tagging_along_with_svm_bert.csv", index = False)